# Python 正则表达式备忘录

* 常用元字符
* re 常用函数
    + re.complile
    + re.match
    + re.search
    + re.findall
    + re.finditer
* 分组、捕获和不捕获
* re 常用特性(两种方式，全称/缩写)
    + re.VERBOSE
    + re.MULTILINE
    + 贪婪和非贪婪
* 环视(Look around)

## 1. 常用元字符

|元字符|含义|
|:----:|----|
|.|匹配除了换行符之外的任何字符。|
|[ ]|字符类别，常用来匹配字符集，字符可以单个列出，也能用`-`表示区间。|
|[^ ]|否定字符类。|
|*|匹配前一个字符的 0 或 更多重复。|
|+|匹配前一个字符的 1 或 更多重复。|
|?|让前一个字符可选|
|{n,m}|对前一个字符，至少匹配 n 次，但不超过 m 次|
|(xyz)|字符组，精确匹配 xyz 组|
|&#124;|或，匹配 &#124; 之前或之后|
|&#92;|转义 `[ ] ( ) { } . * + ? ^ $ \ &`|
|^|匹配最开头的零宽|
|$|匹配最后面的零宽|

## 2. 预设特殊字符

字符|含义
--|:----------------------------
\d|匹配任何十进制数；它相当于类 [0-9]。
\D|匹配任何非数字字符；它相当于类 [^0-9]。
\s|匹配任何空白字符；它相当于类  [ \t\n\r\f\v]。
\S|匹配任何非空白字符；它相当于类 [^ \t\n\r\f\v]。
\w|匹配任何字母数字字符；它相当于类 [a-zA-Z0-9_] 加上中文等。
\W|匹配任何非字母数字字符；它相当于类 [^a-zA-Z0-9_]。

**常用边界符**

字符|含义
----|---
^|匹配一行的开头
$|匹配一行的结尾
\b|匹配一个英文单词的边界或中文句子(全角半角皆可)的边界
\B|\b 的反向
\A|匹配输入的开头
\Z|匹配输入的结尾

## 3. 常用函数和方法

* compile 函数
* match 函数
* search 函数
* findall 函数
* finditer 函数
* split 函数
* sub 函数
* subn 函数
* group 方法

### compile 编译

形式：`complie(pattern[, flag])` pattern 是需要编译的模式，flag 表示可选，如忽略大小写，多行模式等等

In [1]:
import re

p = re.compile('hello')
print(p)

re.compile('hello')


### match 和 search 

match 和 search 的区别在于，match 是从最开头开始匹配，而 search 是全文搜寻，匹配到第一个则听下，无疑 search 用的比 match 更频繁

In [2]:
s = 'Hi, hello world'

print('Match:', p.match(s))
print('Search:', p.search(s))

Match: None
Search: <_sre.SRE_Match object; span=(4, 9), match='hello'>


### split

In [3]:
re.split(r'\n', 'Beautiful is better than ugly.\nExplicit is better than implicit')

['Beautiful is better than ugly.', 'Explicit is better than implicit']

In [4]:
p = re.compile(r'\W')
p.split('Beautiful is better than ugly.', 2)  # 最多做两次切分

['Beautiful', 'is', 'better than ugly.']

### sub 替换

In [5]:
p = re.compile(r'[0-9]+ ')
p.sub(' -> ', 'order0 order1 order13')

'order -> order -> order13'

甚至可以用函数

In [6]:
def normalize_orders(matchobj):
    return 'A' if matchobj.group(1) == '-' else 'B'

re.sub(r'([-|A-Z])', normalize_orders, '-1234 A193 B123')

'A1234 B193 B123'

### findall 和 finditer

两者的区别在于前者返回 list，后者返回可迭代对象

In [7]:
s = 'Hello, world! Hello jack.'
p = re.compile('Hello')
match_all = p.findall(s)
print(match_all)

['Hello', 'Hello']


In [8]:
for i in p.finditer(s):
    print(i.group())

Hello
Hello


### expand

In [20]:
p = re.compile(r'(?P<first>\w+) (?P<second>\w+)')
m = p.search('Hello world') 
m.expand(r'<b>\g<first><b>')  # 比直接 sub 替换更简洁一些

'<b>Hello<b>'

## 4. Grouping

### group([group1,...])

In [9]:
p = re.compile(r'(\w+) (\w+)')  # 这会匹配两个单词组
m = p.search('Hello world')

In [10]:
m.group()

'Hello world'

In [11]:
m.group(0)

'Hello world'

In [12]:
m.group(1)

'Hello'

In [13]:
m.group(2)

'world'

In [14]:
m.group(3)

IndexError: no such group

### groupdict([default])

In [15]:
p = re.compile(r'(?P<first>\w+) (?P<second>\w+)')
p.search('Hello world').groupdict()  # 注意：如果没有命名组，default 则为空

{'first': 'Hello', 'second': 'world'}

### group 其他

In [16]:
p = re.compile(r'(?P<first>\w+) (?P<second>\w+)')
m = p.search('Hello world') # 注意：如果没有命名组，default 则为空
m.start(2) #第二个 group 的 start

6

In [18]:
p = re.compile(r'(?P<first>\w+) (?P<second>\w+)')
m = p.search('Hello world') 
m.end(2) #第二个 group 的 end

11

In [19]:
p = re.compile(r'(?P<first>\w+) (?P<second>\w+)')
m = p.search('Hello world') 
m.span(2) #第二个 group 的 start 和 end

(6, 11)

### 这里有个 python cookbook 3 的例子

In [41]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

In [51]:
for month, day, year in datepat.findall(text):
    print('{}-{}-{}'.format(year, month, day))

2012-11-27
2013-3-13


In [75]:
datepat = re.compile(r'(?P<month>\d+)/(?P<day>\d+)/(?P<year>\d+)')
for m in datepat.finditer(text):
    print('{}-{}-{}'.format(m.group('year'), m.group('month'), m.group('day')))

2012-11-27
2013-3-13


### 分组是很有意义的，比如用在分割上

分组捕获

In [82]:
s = '我是个好人，我真的是个好人。你们不要冤枉我'
p = re.compile(r'(，|。)')
p.split(s)

['我是个好人', '，', '我真的是个好人', '。', '你们不要冤枉我']

分组不捕获

In [89]:
p = re.compile(r'(?:，|。)')
p.split(s)

['我是个好人', '我真的是个好人', '你们不要冤枉我']

用 `[]` 号做可选

In [80]:
p = re.compile(r'[，。]')
p.split(s)

['我是个好人', '我真的是个好人', '你们不要冤枉我']

那如果需要保留符号并添加到一句话的后头呢？

In [85]:
import itertools
p = re.compile(r'(，|。)') 
m = p.split(s)
sentences = m[0::2]
delimiters = m[1::2]
concat = itertools.zip_longest(sentences, delimiters, fillvalue='')
final_sentences = [''.join(i) for i in concat]  # 每个 sentence 和 delimiter
print(final_sentences)

['我是个好人，', '我真的是个好人。', '你们不要冤枉我']


## 5. 其他常用

* re.VERBOSE
* re.MULTILINE

待补

## 6. 环视(Look aroud):

**环视类别：**

标识|中文含义|英文含义
-----------|:--------------:|:----------------
`(?=pattern)`|肯定顺序环视|positive lookahead
`(?!pattern)`|否定顺序环视|negative lookahead
`(?<=pattern)`|肯定逆序环视|positive lookbehind
`(?<!pattern)`|否定逆序环视|negative lookbehind

### 肯定顺序环视：

**环视是无宽度的**

元字符 `^`、`$` 在开头和结尾时候都是零宽度的，而环视也是另宽度的。

In [21]:
pattern = re.compile(r'fox')
result = pattern.search('The quick brown fox jumps over the lazy dog')
print('Without lookahead:')
print(result.start(), result.end(), end='\n\n')

pattern = re.compile(r'(?=fox)')
result = pattern.search('The quick brown fox jumps over the lazy dog')
print('With lookahead:')
print(result.start(), result.end())

Without lookahead:
16 19

With lookahead:
16 16


**环视和非环视匹配**

例：寻找 `,` 号结尾的单词

In [22]:
pattern = re.compile(r'\w+(?=,)')
m = pattern.findall('They were three: Felix, Victor, and Carlos.')
print('With lookahead')
print(m, end='\n\n')

pattern = re.compile(r'\w+,')
m = pattern.findall('They were three: Felix, Victor, and Carlos')
print('Without lookahead')
print(m)

With lookahead
['Felix', 'Victor']

Without lookahead
['Felix,', 'Victor,']


例：寻找 `,` 或 `.` 结尾的单词，但不包含他们

In [23]:
pattern = re.compile(r'\w+(?=,|\.)')
print(pattern.findall('They were three: Felix, Victor, and Carlos.'))

['Felix', 'Victor', 'Carlos']


### 否定顺序环视：

例：找出非 Smith 结尾的 John，并替换为 Sam

In [24]:
pattern = re.compile(r'John(?!\sSmith)')
s = ('I would rather go out with John McLane than with '
     'John Smith or John Bon Jovi.')
result = pattern.finditer(s)
for i in result:
    print(i.start(), i.end())

replacement = re.sub(pattern, 'Sam', s)
print(replacement)

27 31
63 67
I would rather go out with Sam McLane than with John Smith or Sam Bon Jovi.


### 环视和替换：

例：1234567890 -> 1,234,567,890

In [25]:
pattern = re.compile(r'\d{1,3}')
print(pattern.findall('The number is: 12345567890'))

['123', '455', '678', '90']


In [26]:
pattern = re.compile(r'\d{1,3}(?=(\d{3})+(?!\d))')
results = pattern.finditer('1234567890')
for result in results:
    print(result.group())

1
234
567


* `\d{1, 3}` 匹配 1 位或 2 位 或 3 位数字
* `(?=` 肯定顺序环视(注意这里没有组)
* `(\d{3})` 含有 3 个数字的组
* `+` 前面的组至少匹配一次
* `(?!` 否定顺序环视开始
* `\d` 一个数字

In [27]:
pattern = re.compile(r'(\d)(?=(\d{3})+(?!\d))')
s = '12345678900'
print(re.sub(pattern, r'\1,', s))

12,345,678,900


### 肯定逆序环视：

刚刚的数字例子还可以用逆序环视来做

In [28]:
pattern = re.compile(r'(?<=\d)(?=(\d{3})+(?!\d))')
s = '1234567890'
print(re.sub(pattern, ',', s))

1,234,567,890


**逆序环视**在 re 中的问题：

In [29]:
# pattern = re.compile(r'(?<=(John|Jonathan)\s)McLane')  # 这里会提示需要固定宽度

In [30]:
import regex

In [31]:
p = regex.compile(r'(?<=(John|Jonathan)\s)McLane')
s = ('I would rather go out with John McLane'
     'than with John Smith or Jonathan McLane')
for i in p.finditer(s):
    print(i.group()) # groups 和 group 的区别，待补
    
# 小组是从左向右计数的，从1开始。组可以被嵌套。计数的数值可以通过从左到右计算打开的括号数来确定。
# The groups() 方法返回一个包含所有小组字符串的元组，从 1 到 所含的小组号。

McLane
McLane


In [32]:
text = 'Know your Big Data = 5 for $50 on eBooks and 40% off all eBooks until Friday #bigdata #hadoop @HadoopNews packtpub.com/bigdataoffers'
pattern = re.compile(r'(?<=\B@)[\w_]+')
print(pattern.findall(text))

['HadoopNews']


### 否定逆序环视：

In [33]:
pattern = re.compile(r'(?<!John\s)Doe')
results = pattern.finditer('John Doe, Calvin Doe, Hobbes Doe')
for result in results:
    print(result.start(), result.end())

17 20
29 32
